<a href="https://colab.research.google.com/github/SivaHemanth24/cdqa/blob/master/CDQA_for_PDF's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdqa

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 102kB 6.0MB/s 
     |████████████████████████████████| 286kB 13.7MB/s 
     |████████████████████████████████| 10.5MB 19.0MB/s 
     |████████████████████████████████| 317kB 55.2MB/s 
     |████████████████████████████████| 6.7MB 24.8MB/s 
     |████████████████████████████████| 748.9MB 24kB/s 
     |████████████████████████████████| 92kB 13.6MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 1.0MB 41.5MB/s 
     |████████████████████████████████| 890kB 36.6MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp36-none-any.whl size=47640 sha256=66c2c1f88fa1994c8f3d905608a6a2d13a85b4d2466bcfd8b3473ea06f90f567
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9e946bf5a8cf922e1c86fb6b8a33d9
  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29222 sha256=cdc2460ee7112dbd20b6b5ec80e11931198d11d25dc8248a315e54

In [2]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')

In [4]:
# Download pdf files from BNP Paribas public news
def download_pdf():
    import os
    import wget
    directory = './data/pdf/'
    models_url = [
      'https://invest.bnpparibas.com/documents/1q19-pr-12648',
      'https://invest.bnpparibas.com/documents/4q18-pr-18000',
      'https://invest.bnpparibas.com/documents/4q17-pr'
    ]

    print('\nDownloading PDF files...')

    if not os.path.exists(directory):
        os.makedirs(directory)
    for url in models_url:
        wget.download(url=url, out=directory)

download_pdf()

In [5]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

2020-04-14 16:53:02,440 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2020-04-14 16:53:03,028 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2020-04-14 16:53:03,520 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


ImportError: ignored

           title                                         paragraphs
0  1q19-pr-12648  [FIRST QUARTER 2019 RESULTS   PRESS RELEASE Pa...
1        4q17-pr  [2017 FULL YEAR RESULTS   PRESS RELEASE Paris,...
2       4q18-pr2  [2018 FULL YEAR RESULTS   PRESS RELEASE Paris,...

In [6]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 674093.57B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [0]:
query = 'BNP Paribas established in the year?'
prediction = cdqa_pipeline.predict(query)

In [10]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: BNP Paribas established in the year?
answer: 2018
title: 4q18-pr2
paragraph: BNP Paribas’ financial disclosures for the fourth quarter 2018 and for the year 2018 are contained in this press release and in the presentation attached herewith. All legally required disclosures, including the Registration document, are available online at http://invest.bnpparibas.com in the “Results” section and are made public by BNP Paribas pursuant to the requirements under Article L.451-1-2 of the French Monetary and Financial Code and Articles 222-1 et seq. of the Autorité des Marchés Financiers’ general rules.   
